In [1]:
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [2]:
opt = {'n_epochs':200,'batch_size':64,'lr':0.0002,'b1':0.5,'b2':0.999,'n_cpu':8,'latent_dim':100,'img_size':28,'channels':1,'sample_interval':400}

In [3]:
img_shape = (opt['channels'],opt['img_size'],opt['img_size'])

In [4]:
cuda = True if torch.cuda.is_available() else False

In [5]:
print(cuda)

True


In [6]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator,self).__init__()
        
        def block(in_feat, out_feat, normalize = True):
            layers = [nn.Linear(in_feat,out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat,0.8))
            layers.append(nn.LeakyReLU(0.2,inplace=True))
            return layers
        
        self.model = nn.Sequential(
            *block(opt['latent_dim'],128,normalize=False),
            *block(128,256),
            *block(256,512),
            *block(512,1024),
            *block(1024,int(np.prod(img_shape))),
            nn.Tanh()
        )
    
    def forward(self,z):
        img = self.model(z)
        img = img.view(img.size(0),*img_shape)
        return img

In [7]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator,self).__init__()
        
        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)),512),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Linear(512,256),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Linear(256,1),
            nn.Sigmoid()
        )
    
    def forward(self,img):
        img_flag = img.view(img.size(0), 1)
        validity = self.model(img_flag)
        
        return validity

In [8]:
loss = torch.nn.BCELoss()

generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda();
    loss.cuda()

    os.makedirs("../GANs/MNIST",exist_ok=True)

In [13]:
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../GANs/MNIST",
        train=True,
        download = False,
        transform = transforms.Compose(
            [transforms.Resize(opt['img_size']),transforms.ToTensor(),transforms.Normalize([0.5],[0.5])]
        )
    ),
    batch_size=opt['batch_size'],
    shuffle=True
)

In [14]:
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt['lr'], betas=(opt['b1'],opt['b2']))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt['lr'], betas=(opt['b1'],opt['b2']))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [15]:
for epoch in range(opt['n_epochs']):
    for i, (img,_) in enumerate(dataloader):
        
        valid = Variable(Tensor(img.size(0), 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(img.size(0), 1).fill_(0.0), requires_grad=False)
        
        real_imgs = Variable(img.type(Tensor))
        
        optimizer_G.zero_grad()
        
        z = Variable(Tensor(np.random.normal(0.,1,(img.shape[0], opt['latent_dim']))))
        
        gen_imgs = generator(z)
        
        g_loss = loss(discriminator(gen_imgs), valid)
        
        g_loss.backward()
        optimizer_G.step()
        
        optimizer_D.zero_grad()
        
        real_loss = loss(discriminator(real_imgs), valid)
        fake_loss = loss(discriminator(gen_imgsen), fake)
        
        d_loss.backward()
        optimizer_D.step()
        
        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, opt['n_epochs'], i, len(dataloader), d_loss.item(), g_loss.item())
        )
        
        batches_done = epoch * len(dataloader) + i
            
        if batches_done % opt['sample_interval'] == 0:
            save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)

RuntimeError: shape '[64, 1]' is invalid for input of size 50176